In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime

In [2]:
engine = create_engine("sqlite:///hashtag2020.db", echo=True)
conn = engine.connect()

2021-02-03 19:02:42,855 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-03 19:02:42,859 INFO sqlalchemy.engine.base.Engine ()
2021-02-03 19:02:42,868 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-03 19:02:42,872 INFO sqlalchemy.engine.base.Engine ()


In [3]:
#Create the table. If we use automap_base, the table must have a primary key
conn.execute("""
    CREATE TABLE IF NOT EXISTS tweet_counts (
        created_at PRIMARY KEY
        tweet VARCHAR(280)
        likes integer
        retweet_count integer
        source VARCHAR(30)
        user_location VARCHAR(30)
        lat integer
        long integer
    )
""")

2021-02-03 19:02:42,945 INFO sqlalchemy.engine.base.Engine 
    CREATE TABLE IF NOT EXISTS tweet_counts (
        created_at PRIMARY KEY
        tweet VARCHAR(280)
        likes integer
        retweet_count integer
        source VARCHAR(30)
        user_location VARCHAR(30)
        lat integer
        long integer
    )

2021-02-03 19:02:42,949 INFO sqlalchemy.engine.base.Engine ()
2021-02-03 19:02:42,956 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near "tweet": syntax error
[SQL: 
    CREATE TABLE IF NOT EXISTS tweet_counts (
        created_at PRIMARY KEY
        tweet VARCHAR(280)
        likes integer
        retweet_count integer
        source VARCHAR(30)
        user_location VARCHAR(30)
        lat integer
        long integer
    )
]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [4]:
trump_csv_path = "../hashtag_donaldtrump.csv"
biden_csv_path = "../hashtag_joebiden.csv"

In [5]:
trump_df = pd.read_csv(trump_csv_path, lineterminator='\n')
biden_df = pd.read_csv(biden_csv_path, lineterminator='\n')

In [6]:
trump_df.reset_index(inplace=True)
trump_df.head()

,index,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,1,2020-10-15 00:00:01,1.316529e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,3.316176e+08,Tgcom24,MediasetTgcom24,...,1067661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530
2,2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,...,1185.0,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
3,3,2020-10-15 00:00:02,1.316529e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,8.283556e+17,Trumpytweeter,trumpytweeter,...,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591
4,4,2020-10-15 00:00:08,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,4.741380e+07,Rana Abtar - رنا أبتر,Ranaabtar,...,5393.0,Washington DC,38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121


In [7]:
trump_data_df = trump_df[['created_at', 'tweet', 'likes', 'retweet_count', 'source','user_location', 'lat', 'long', 'city','state','state_code','country']]

In [8]:
#trump_data_df.isnull().values.sum()

In [9]:
# trump_null= trump_data_df[trump_data_df.isna().any(axis=1)]
# trump_null

In [10]:
trump_dropna_df = trump_data_df.dropna(how='any')
trump_dropna_df.head()

,created_at,tweet,likes,retweet_count,source,user_location,lat,long,city,state,state_code,country
2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,Portland,45.520247,-122.674195,Portland,Oregon,OR,United States of America
4,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,Washington DC,38.894992,-77.036558,Washington,District of Columbia,DC,United States of America
11,2020-10-15 00:00:25,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,Twitter for iPhone,"Chicago, Illinois",41.875562,-87.624421,Chicago,Illinois,IL,United States of America
12,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,Twitter for Android,"San Diego, CA",32.717421,-117.162771,San Diego,California,CA,United States of America
13,2020-10-15 00:00:31,@Susan_Hutch @JoeBiden #Ukraine @RealDonaldTru...,2.0,0.0,Twitter Web App,Edinburgh ~ Heart of Scotland!,55.953346,-3.188375,City of Edinburgh,Scotland,SCT,United Kingdom


In [11]:
# trump_dropna_df['new_date'] = [d.date() for d in trump_dropna_df['created_at']]
# trump_dropna_df['new_time'] = [d.time() for d in trump_dropna_df['created_at']]
# trump_dropna_df.head()

trump_dropna_df['created_at'] = pd.to_datetime(trump_dropna_df['created_at'])
trump_dropna_df['Date'] = trump_dropna_df['created_at'].dt.strftime('%m/%d/%Y')
trump_dropna_df['Time'] = trump_dropna_df['created_at'].dt.strftime('%H:%M:%S')
trump_dropna_df.head()



/Users/buckleyweglarz/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/buckleyweglarz/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/buckleyweglarz/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,created_at,tweet,likes,retweet_count,source,user_location,lat,long,city,state,state_code,country,Date,Time
2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,Portland,45.520247,-122.674195,Portland,Oregon,OR,United States of America,10/15/2020,00:00:02
4,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,Washington DC,38.894992,-77.036558,Washington,District of Columbia,DC,United States of America,10/15/2020,00:00:08
11,2020-10-15 00:00:25,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,Twitter for iPhone,"Chicago, Illinois",41.875562,-87.624421,Chicago,Illinois,IL,United States of America,10/15/2020,00:00:25
12,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,Twitter for Android,"San Diego, CA",32.717421,-117.162771,San Diego,California,CA,United States of America,10/15/2020,00:00:26
13,2020-10-15 00:00:31,@Susan_Hutch @JoeBiden #Ukraine @RealDonaldTru...,2.0,0.0,Twitter Web App,Edinburgh ~ Heart of Scotland!,55.953346,-3.188375,City of Edinburgh,Scotland,SCT,United Kingdom,10/15/2020,00:00:31


In [14]:
trump_dropna_df.reset_index(level=0, inplace=False)
trump_dropna_df.head()
#trump_dropna_df.value_counts("created_at")

,created_at,tweet,likes,retweet_count,source,user_location,lat,long,city,state,state_code,country,Date,Time
2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,Portland,45.520247,-122.674195,Portland,Oregon,OR,United States of America,10/15/2020,00:00:02
4,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,Washington DC,38.894992,-77.036558,Washington,District of Columbia,DC,United States of America,10/15/2020,00:00:08
11,2020-10-15 00:00:25,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,Twitter for iPhone,"Chicago, Illinois",41.875562,-87.624421,Chicago,Illinois,IL,United States of America,10/15/2020,00:00:25
12,2020-10-15 00:00:26,#Trump #PresidentTrump #Trump2020LandslideVict...,3.0,5.0,Twitter for Android,"San Diego, CA",32.717421,-117.162771,San Diego,California,CA,United States of America,10/15/2020,00:00:26
13,2020-10-15 00:00:31,@Susan_Hutch @JoeBiden #Ukraine @RealDonaldTru...,2.0,0.0,Twitter Web App,Edinburgh ~ Heart of Scotland!,55.953346,-3.188375,City of Edinburgh,Scotland,SCT,United Kingdom,10/15/2020,00:00:31


In [ ]:
# Insert data into DB
df.to_sql("tweet_counts", conn, if_exists="append", index=False)